In [1]:
# !pip install trulens-eval
# !pip install pysqlite3
# !pip install ipytree

In [2]:
import sys
sys.path.append('../docker-lambda-aws/image/src/')

import logging
from ipytree import Tree, Node
from dotenv import load_dotenv, find_dotenv
import os
import openai
from trulens_eval import TruChain, Tru
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
from trulens_eval.app import App
from trulens_eval.feedback import Groundedness
import numpy as np
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

from retriever import get_retriever
import config
from curriebot import get_agent_executor


In [3]:
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
_ = load_dotenv(find_dotenv()) 

openai.api_key  = os.environ['OPENAI_API_KEY']

tru = Tru()
tru.reset_database()

INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


# Utility functions 

Reference: https://www.trulens.org/trulens_eval/getting_started/quickstarts/langchain_quickstart/#load-documents

In [5]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))
    
    calls_node = Node('Calls')
    tree.add_node(calls_node)
    
    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)
        
        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)
    
    return tree


# Evaluate RAG 

In [6]:
llm = ChatOpenAI(model=config.model_name, temperature=config.temperature)
retriever = get_retriever()
output_parser = StrOutputParser()
retrieval_resume_template = ChatPromptTemplate.from_messages([
    ("system", """Your task is to answer questions about the resume provided.
                  Answer the question based only on the following context.
                  If you cannot answer the question or there are no resume context provided 
                  tell the user to message `https://www.linkedin.com/in/patricklarosa/`

                  [RESUME CONTEXT]
                  {context}
                  """),
    ("human", "{question}")
])

In [8]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | retrieval_resume_template
    | llm
    | StrOutputParser()
)

In [9]:
provider = OpenAI()

In [10]:
context = App.select_context(chain)

In [11]:
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)


✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets .


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
tru_recorder = TruChain(chain,
    app_id='ResumeRetrieverApplication',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [13]:
with tru_recorder as recording:
    llm_response = chain.invoke("Can you share his work experience?")
    llm_response = chain.invoke("Does he have experience with generative ai?")
    llm_response = chain.invoke("What is his latest employment?")
    llm_response = chain.invoke("What is his first job?")
    llm_response = chain.invoke("total years of experience?")
    llm_response = chain.invoke("What is his professional summary?")
    llm_response = chain.invoke("What is his highest educational attainment?")
    llm_response = chain.invoke("When is his birthday?")
    llm_response = chain.invoke("When is his mobile number?")
    llm_response = chain.invoke("When are his hobbies?")


Groundedness per statement in source:   0%|          | 0/13 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
recs = recording.records 

records, feedback = tru.get_records_and_feedback(app_ids=["ResumeRetrieverApplication"])

records.head(10)

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,context_relevance_with_cot_reasons,groundedness_measure_with_cot_reasons,relevance_calls,context_relevance_with_cot_reasons_calls,groundedness_measure_with_cot_reasons_calls,latency,total_tokens,total_cost
0,ResumeRetrieverApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_b727a00d3855f32b55678a653a4a192f,"""Can you share his work experience?""","""Certainly! Here's a summary of the work exper...",-,"{""record_id"": ""record_hash_b727a00d3855f32b556...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-05-14T14:22:28.171547"", ""...",2024-05-14T14:23:01.413212,1.0,0.9,0.653846,[{'args': {'prompt': 'Can you share his work e...,[{'args': {'question': 'Can you share his work...,[{'args': {'source': [[{'page_content': '- Rec...,33,2167,0.05141
1,ResumeRetrieverApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_bf8ca5f284718dbf6fc4f0caebc171ea,"""Does he have experience with generative ai?""","""Yes, he has experience with generative AI. He...",-,"{""record_id"": ""record_hash_bf8ca5f284718dbf6fc...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-05-14T14:23:01.813168"", ""...",2024-05-14T14:23:11.056413,0.8,0.9,0.000000,[{'args': {'prompt': 'Does he have experience ...,[{'args': {'question': 'Does he have experienc...,[{'args': {'source': [[{'page_content': '- Rec...,9,1815,0.04087
2,ResumeRetrieverApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_32117b90f955318c0a1d982c251850d8,"""What is his latest employment?""","""The latest employment is at TDCX, where the i...",-,"{""record_id"": ""record_hash_32117b90f955318c0a1...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-05-14T14:23:11.475942"", ""...",2024-05-14T14:23:18.797011,1.0,1.0,0.000000,[{'args': {'prompt': 'What is his latest emplo...,[{'args': {'question': 'What is his latest emp...,[{'args': {'source': [[{'page_content': '## Ca...,7,1753,0.03913
3,ResumeRetrieverApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_61030784c623d05423e7b2d1984d7240,"""What is his first job?""","""The first job mentioned in the resume is the ...",-,"{""record_id"": ""record_hash_61030784c623d05423e...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-05-14T14:23:19.206538"", ""...",2024-05-14T14:23:24.755373,1.0,0.8,0.000000,"[{'args': {'prompt': 'What is his first job?',...",[{'args': {'question': 'What is his first job?...,[{'args': {'source': [[{'page_content': 'Busin...,5,1813,0.03901
4,ResumeRetrieverApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_60a8a754d52f9ae639533abd2bf4a5e0,"""total years of experience?""","""Based on the resume provided, Patrick La Rosa...",-,"{""record_id"": ""record_hash_60a8a754d52f9ae6395...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-05-14T14:23:25.129293"", ""...",2024-05-14T14:23:30.275122,1.0,0.8,0.000000,[{'args': {'prompt': 'total years of experienc...,[{'args': {'question': 'total years of experie...,[{'args': {'source': [[{'page_content': 'Busin...,5,1799,0.03886
5,ResumeRetrieverApplication,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_9c817439e3e1d0a104d18d8b3fbd6ac8,"""What is his professional summary?""","""Patrick is an engineer by education, a data s...",-,"{""record_id"": ""record_hash_9c817439e3e1d0a104d...","{""n_requests"": 3, ""n_successful_requests"": 3, ...","{""start_time"": ""2024-05-14T14:23:30.589785"", ""..

In [15]:
tru.get_leaderboard(app_ids=["ResumeRetrieverApplication"])

,relevance,context_relevance_with_cot_reasons,groundedness_measure_with_cot_reasons,latency,total_cost
app_id,,,,,
ResumeRetrieverApplication,0.966667,0.844444,0.461538,8.8,0.037663
